In [ ]:
# Usamos una biblioteca de google para poder acceder a drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Para cambiar mi directorio 
%cd '/content/drive/My Drive/Colab Notebooks/Deep_Learning_Pytorch/Red_generativa_Cycle_Gan/'

# Vemos lo que esta ahi dentro
!ls

/content/drive/My Drive/Colab Notebooks/Deep_Learning_Pytorch/Red_generativa_Cycle_Gan
dl-pytorch  PyTorch-CycleGAN  Untitled0.ipynb


In [ ]:
!git clone https://github.com/aitorzip/PyTorch-CycleGAN 

fatal: destination path 'PyTorch-CycleGAN' already exists and is not an empty directory.


In [ ]:
!ls

dl-pytorch  PyTorch-CycleGAN  Untitled0.ipynb


In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/Deep_Learning_Pytorch/Red_generativa_Cycle_Gan/PyTorch-CycleGAN/'

/content/drive/My Drive/Colab Notebooks/Deep_Learning_Pytorch/Red_generativa_Cycle_Gan/PyTorch-CycleGAN


In [ ]:
%%sh
sh ./download_dataset summer2winter_yosemite

Archive:  ./datasets/summer2winter_yosemite.zip
   creating: ./datasets/summer2winter_yosemite/
   creating: ./datasets/summer2winter_yosemite/trainA/
  inflating: ./datasets/summer2winter_yosemite/trainA/2014-07-01 10:31:48.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2015-07-01 23:43:46.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2014-07-27 21:50:52.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2016-09-12 21:34:08.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2015-06-10 23:35:46.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2016-07-12 13:57:26.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2014-05-13 02:35:47.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2015-07-12 20:23:26.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2015-08-26 00:23:12.jpg  
  inflating: ./datasets/summer2winter_yosemite/trainA/2011-05-19 11:26:37.jpg  
  inflating: ./datasets/summer2winter_yosemite/tr

./download_dataset: 5: ./download_dataset: [[: not found
for details.

--2021-05-06 03:12:19--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/summer2winter_yosemite.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132640856 (126M) [application/zip]
Saving to: ‘./datasets/summer2winter_yosemite.zip’

     0K .......... .......... .......... .......... ..........  0% 1.44M 88s
    50K .......... .......... .......... .......... ..........  0% 3.04M 65s
   100K .......... .......... .......... .......... ..........  0% 2.75M 58s
   150K .......... .......... .......... .......... ..........  0% 2.98M 54s
   200K .......... .......... .......... .......... ..........  0% 35.7M 44s
   250K .......... .......... .......... .......... ..........  0% 3.70M 42s
   300K .......... .........

In [ ]:
!mv datasets/summer2winter_yosemite /content/drive/My\ Drive/Colab\ Notebooks/Deep_Learning_Pytorch/Red_generativa_Cycle_Gan/dl-pytorch/datasets/ 

In [1]:
!ls /content/drive/My\ Drive/Colab\ Notebooks/Deep_Learning_Pytorch/Red_generativa_Cycle_Gan/dl-pytorch/datasets/ 

ls: cannot access '/content/drive/My Drive/Colab Notebooks/Deep_Learning_Pytorch/Red_generativa_Cycle_Gan/dl-pytorch/datasets/': No such file or directory


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
  def __init__(self, in_features):
    super(ResidualBlock, self).__init__()
    
    conv_block = [ nn.ReflectionPad2d(1), #mejor padding
                   nn.Conv2d(in_features, in_features, 3),
                   nn.InstanceNorm2d(in_features), #BN para GANS
                   nn.ReLU(True),
                   nn.ReflectionPad2d(1), #mejor para consrvar distribucion
                   nn.Conv2d(in_features, in_features, 3),
                   nn.InstanceNorm2d(in_features)
                 ]
    
    self.conv_block = nn.Sequential(*conv_block)
    def forward(self, x):
      return self.conv_block(x) + x #una idea poderosa

In [5]:
class Generator(nn.Module):
  def __init__(self, input_nc, output_nc, n_residual_blocks=9):
    super(Generator,self).__init__()
    
    # Bloqueconvolucional
    model = [ nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, 64, F), # I - 7 + 6 /1 +1 = I
            nn.InstanceNorm2d(64),
             nn.ReLU(True)
            ]
    
    in_features = 64
    out_features = in_features * 2
    
    #Encoding
    for _ in range(2):
      model += [ nn.Conv2d(in_features, out_features, 3, stride=2, padding=1), #I/2
                 nn.InstanceNorm2d(out_features),
                 nn.ReLU(True)
               ]
      in_features = out_features
      out_features = in_features*2
    
    #transformaciones residuales
    
    for _ in range(n_residual_blocks):
      model += [ResidualBlock(in_features)]
    
    #decoding
    
    out_features = in_features/2
    for _ in range(2):
      model += [ nn.ConvTransose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1), #2I
                 nn.InstanceNorm2d(out_features),
                 nn.ReLU(True)
               ]
      in_features = out_features
      out_feature = in_features //2
      
      #salida
      model += [ nn.ReflectionPad2d(3),
                 nn.Conv2d(64, output_nc, 7), #I
                 nn.Tanh()
               ]
      
      self.model = nn.Sequential(*model)
      
    def forward(self,x):
      return self.model(x)

In [7]:
class Discriminator(nn.Module):
  "PatchGAN: discrimina estilo o textura"
  def __init__(self, input_nc):
    super(Discriminator, self).__init__()
    
    model = [ nn.Conv2d(input_nc, 64, 4, stride=2, padding=1), #I/2
              nn.LeakyReLU(0.2, in_place=True)
            ]
    
    model += [ nn.Conv2d(64, 128, 4, stride=2, padding=1), #I/2
               nn-InstanceNorm2d(128),
              nn.LeakyReLU(0.2, in_place=True)
             ]
    
    model += [ nn.Conv2d(128, 256, 4, stride=2, padding=1), #I/2
               nn-InstanceNorm2d(256),
              nn.LeakyReLU(0.2, in_place=True)
             ]
    
    model += [ nn.Conv2d(256, 512, 4, padding=1), #I-1
               nn-InstanceNorm2d(512),
              nn.LeakyReLU(0.2, in_place=True)
             ]
    
    # Flatten
    model += [nn.Conv2d(512, 1, 4, padding=1)] #I-1
    
    self.model == nn.Sequential(*model)
    
  def forward(self, x):
    x = self.model(x)
    return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1)